In [175]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [105]:
df1 = pd.read_csv('data/features_basicas.csv', low_memory=False, index_col='person')
#df1["proprosion"] = np.where(df1["cant_viewed_product"]>0, (df1["cant_checkouts"]/df1["cant_viewed_product"]),0)


In [106]:
df2 = pd.read_csv('data/vistas_promedio.csv', low_memory=False, index_col='person')
#df3 = pd.read_csv('data/featureUsuarioRealizaAlMenos10EventosEn20Minutos.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('data/features_mas_vistos.csv', low_memory=False, index_col='person')
df5 = pd.read_csv('data/features_compras.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('data/features_checkout.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('data/features_time.csv', low_memory=False, index_col='person')
df8 = pd.read_csv('data/features_time_2.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('data/features_time_3.csv', low_memory=False, index_col='person')

df_unidos = df1.join(df2).fillna(0)
#df_unidos = df_unidos.join(df3).fillna(0)
df_unidos = df_unidos.join(df4).fillna(0)
df_unidos = df_unidos.join(df5).fillna(0)
df_unidos = df_unidos.join(df6).fillna(0)
df_unidos = df_unidos.join(df7).fillna(0)
df_unidos = df_unidos.join(df8).fillna(0)
df_unidos = df_unidos.join(df9).fillna(0)
df_unidos


,cant_conversions,cant_checkouts,cant_viewed_product,cant_searched_product,cant_ad_campaign_hit,ad_campaign_hit,cant_lead,cant_brand_listing,brand listing,total_sesiones,...,vio_mas_motorola,compro_apple,compro_samsung,compro_motorola,check_apple,check_samsung,check_motorola,timestamp,prom_timestamp,sum_timestamp
person,,,,,,,,,,,,,,,,,,,,,
4886f805,0.0,1.0,4.0,1.0,0.0,False,0.0,0.0,False,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1526603430000000000,1526602376249999872,1.221282e+19
ad93850f,0.0,1.0,20.0,0.0,10.0,True,0.0,15.0,True,5,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1527029454000000000,1526573415320754944,8.090839e+19
0297fc1e,0.0,7.0,404.0,6.0,29.0,True,1.0,4.0,True,89,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1527508212000000000,1522498682203065856,7.947443e+20
2d681dd8,0.0,1.0,13.0,1.0,1.0,True,0.0,5.0,True,2,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1527450169000000000,1527141699500000256,3.359712e+19
cccea85e,0.0,1.0,739.0,1.0,15.0,True,0.0,7.0,True,17,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1527777510000000000,1526707378753567488,1.177091e+21
4c8a8b93,0.0,2.0,177.0,9.0,14.0,True,0.0,8.0,True,14,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1526949198000000000,1526693704236841472,3.480862e+20
1b9f7cf6,0.0,1.0,9.0,8.0,0.0,False,0.0,3.0,True,3,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1526897630000000000,1526618185000000256,3.511222e+19
29ebb414,0.0,1.0,8.0,11.0,4.0,True,0.0,0.0,False,4,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1526837857000000000,1526651492240000256,3.816629e+19
de8fe91b,0.0,1.0,27.0,13.0,0.0,False,0.0,3.0,True,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1526942248000000000,1526727758057692672,7.938984e+19


In [107]:
labels = pd.read_csv('data/labels_training_set.csv', low_memory=False, index_col='person')

df_test = df_unidos.join(labels, how= "inner")
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19414 entries, ad93850f to d8d6b0da
Data columns (total 25 columns):
cant_conversions               19414 non-null float64
cant_checkouts                 19414 non-null float64
cant_viewed_product            19414 non-null float64
cant_searched_product          19414 non-null float64
cant_ad_campaign_hit           19414 non-null float64
ad_campaign_hit                19414 non-null bool
cant_lead                      19414 non-null float64
cant_brand_listing             19414 non-null float64
brand listing                  19414 non-null bool
total_sesiones                 19414 non-null int64
promedio_eventos_por_sesion    19414 non-null float64
mas_prom                       19414 non-null float64
vio_mas_apple                  19414 non-null float64
vio_mas_samsung                19414 non-null float64
vio_mas_motorola               19414 non-null float64
compro_apple                   19414 non-null float64
compro_samsung                

In [108]:
features = df_test.columns.tolist()
features.remove('label')
print(features)

['cant_conversions', 'cant_checkouts', 'cant_viewed_product', 'cant_searched_product', 'cant_ad_campaign_hit', 'ad_campaign_hit', 'cant_lead', 'cant_brand_listing', 'brand listing', 'total_sesiones', 'promedio_eventos_por_sesion', 'mas_prom', 'vio_mas_apple', 'vio_mas_samsung', 'vio_mas_motorola', 'compro_apple', 'compro_samsung', 'compro_motorola', 'check_apple', 'check_samsung', 'check_motorola', 'timestamp', 'prom_timestamp', 'sum_timestamp']


In [109]:
labels['label'].value_counts(normalize=True)

0    0.949521
1    0.050479
Name: label, dtype: float64

In [110]:
rf = RandomForestClassifier(n_estimators=10000, n_jobs=-1, min_samples_split=150,\
                            class_weight='balanced')

#  Partimos los datos que tenemos para entrenar en dos partes, una para entrenar el modelo (80% de los datos)
#  y la otra parte se usará para probar el modelo (el 20% restante)
X_train, X_test, Y_train, Y_test = train_test_split(df_test[features],\
                                                    df_test['label'], test_size=0.20)

In [111]:
rf.fit(X_train,Y_train)
rf.score(X_test,Y_test)

0.80916816894154009

In [112]:
Y_pred = rf.predict(X_test)
print(roc_auc_score(Y_test, Y_pred))

0.784087327737


In [113]:
#  Esto nos tira data sobre la precisión, cuántas de las cosas que clasificamos salieron bien clasificadas, y f1

Y_truths = Y_test
print(classification_report(Y_truths, Y_pred))

             precision    recall  f1-score   support

          0       0.98      0.81      0.89      3674
          1       0.19      0.76      0.30       209

avg / total       0.94      0.81      0.86      3883



In [114]:
print(confusion_matrix(Y_truths, Y_pred))

[[2984  690]
 [  51  158]]


In [115]:
feature_importances = pd.DataFrame(rf.feature_importances_,\
                                   index = X_train.columns,\
                                    columns=['importance']).sort_values('importance',ascending=False)

In [116]:
feature_importances.head(50)

,importance
cant_checkouts,0.280408
prom_timestamp,0.124492
timestamp,0.120228
sum_timestamp,0.073370
cant_viewed_product,0.052419
total_sesiones,0.046784
check_samsung,0.043337
check_apple,0.041280
cant_brand_listing,0.035275
promedio_eventos_por_sesion,0.034190


## Summit

In [117]:
df_submit = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos, how='inner')

In [118]:
df_events.head()

,cant_conversions,cant_checkouts,cant_viewed_product,cant_searched_product,cant_ad_campaign_hit,ad_campaign_hit,cant_lead,cant_brand_listing,brand listing,total_sesiones,...,vio_mas_motorola,compro_apple,compro_samsung,compro_motorola,check_apple,check_samsung,check_motorola,timestamp,prom_timestamp,sum_timestamp
person,,,,,,,,,,,,,,,,,,,,,
4886f805,0.0,1.0,4.0,1.0,0.0,False,0.0,0.0,False,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1526603430000000000,1526602376249999872,1.221282e+19
0297fc1e,0.0,7.0,404.0,6.0,29.0,True,1.0,4.0,True,89,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1527508212000000000,1522498682203065856,7.947443e+20
2d681dd8,0.0,1.0,13.0,1.0,1.0,True,0.0,5.0,True,2,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1527450169000000000,1527141699500000256,3.359712e+19
cccea85e,0.0,1.0,739.0,1.0,15.0,True,0.0,7.0,True,17,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1527777510000000000,1526707378753567488,1.177091e+21
4c8a8b93,0.0,2.0,177.0,9.0,14.0,True,0.0,8.0,True,14,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1526949198000000000,1526693704236841472,3.480862e+20


In [119]:
kaggle_pred = rf.predict_proba(df_events)
kaggle_pred
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

In [120]:
df_submit.to_csv('submit.csv')


In [121]:
df_submit.label.mean()

0.2701889301610269

In [73]:
0.34460859613130107

0.34460859613130107